### קוד מבוא

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 

import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [4]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [5]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [6]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [7]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [8]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


In [9]:

def split_index_by_taz(index,taz,min_prec,col_name_to_split):
    index['index_area']=index.area
    
    taz['taz_area']=taz.area

    index_taz=index.overlay(taz[['Taz_num','taz_area','geometry']])

    index_taz['small_area']=index_taz.area

    index_taz['precent_from_big_index']=index_taz['small_area']/index_taz['index_area']
    
    index_taz['precent_from_big_taz']=index_taz['small_area']/index_taz['taz_area']

    index_taz=index_taz.loc[(index_taz['precent_from_big_index']>min_prec)|(index_taz['precent_from_big_taz']>0.9)]
    
    index_taz=index_taz[['id','Taz_num','precent_from_big_index']]

    new_big=index_taz.groupby(['id']).sum()

    index=index.set_index('id')
    index['new_big']=new_big['precent_from_big_index']

    index=pd.merge(index.reset_index(),index_taz,on='id')

    for c in col_name_to_split:
        index['{}'.format(c)]=index['{}'.format(c)]*(index['precent_from_big_index']/index['new_big'])
        
        
    return index

### העלת קבצים

In [10]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]


#### קיבולת ברמת יישוב (חלקי) ביו"ש

In [11]:
path=r'{}\background_files'.format(software_data_folder_location)

In [12]:
kibolt_yosh_some_towns=up_load_df(path,'230628_Jewish_kibolt_some_towns_in_yosh_till_2050')

In [13]:
col=[ 'Taz_num', 'add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

In [14]:
kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns.pivot_table(index='Taz_num',aggfunc=sum).reset_index()[col]

In [15]:
kibolt_yosh_some_towns_sum_by_taz['kibolet_type']='Shrinkable'

In [16]:
old_col=[ 'Taz_num','add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050','kibolet_type']

new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns_sum_by_taz[old_col]

kibolt_yosh_some_towns_sum_by_taz.columns=new_col

In [17]:
kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns_sum_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

In [18]:
kibolt_yosh_some_towns_sum_by_taz['source']='by_town'

מעקב אחרי מקורות קיבולת לכל TAZ

In [20]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

In [21]:
taz.loc[taz['Taz_num'].isin(list(kibolt_yosh_some_towns_sum_by_taz.index)),'by_town']=1

#### קיבולת ברמת אזור תנועה (חלקי)

In [22]:
Jewish_kibolt_some_by_taz=up_load_shp(r'{}\background_files\kibolt_jew_by_some_taz_230628.shp'.format(software_data_folder_location))

In [23]:
old_col=[ 'Taz_num','add_aprt_2',
 'add_aprt_3',
 'add_aprt_4',
 'add_aprt_5',
 'add_aprt_6',
 'add_aprt_7','type_kibol']

In [24]:
new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

In [25]:
Jewish_kibolt_some_by_taz=Jewish_kibolt_some_by_taz[old_col]

In [26]:
Jewish_kibolt_some_by_taz.columns=new_col

In [27]:
Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==1,'kibolet_type']='Shrinkable'

In [28]:
Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==2,'kibolet_type']='left_the_station'

In [29]:
Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==3,'kibolet_type']='Depends_on_policy'

In [30]:
Jewish_kibolt_some_by_taz=Jewish_kibolt_some_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

In [31]:
Jewish_kibolt_some_by_taz['source']='by_taz'

מעקב אחרי מקורות קיבולת לכל TAZ

In [32]:
taz.loc[taz['Taz_num'].isin(list(Jewish_kibolt_some_by_taz.Taz_num)),'by_taz']=1

#### קיבולת ברמת תכנית עם אזורי תנועה בלבד (חלקי)

In [33]:
Jewish_kibolt_some_by_plan_only_with_taz=up_load_shp(r'{}\background_files\Jewish_kibolt_some_by_plan_with_taz_num.shp'.format(software_data_folder_location))
Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz[['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_ty']].rename(columns={'kibolet_ty':'kibolet_type'})

Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

In [34]:
Jewish_kibolt_some_by_plan_only_with_taz['source']='plan_only_with_taz'

מעקב אחרי מקורות קיבולת לכל TAZ

In [35]:
taz.loc[taz['Taz_num'].isin(list(Jewish_kibolt_some_by_plan_only_with_taz.Taz_num)),'plan_only_with_taz']=1

#### קיבולת ברמת תכנית  (חלקי)

In [37]:
df=up_load_df(r'{}\background_files'.format(software_data_folder_location),'index_for_id_polygons_230717')

c:\Users\gidon\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [38]:
shp=up_load_shp(r'{}\background_files\230628_id_polygons_for_index.shp'.format(software_data_folder_location))


index=shp.merge(df,on='id',how='outer')

#לחלק את התכניות לפי אזורי תנועה

index=split_index_by_taz(index,taz,0.25,['add_aprt'])

#לייצר קיבולת כל חומש

years=[ '2025', '2030', '2035', '2040', '2045', '2050']

index[years]=index[years].apply(lambda x: x * index['risk_factor']*index['add_aprt'])

col=['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

Jewish_kibolt_some_by_plan=index[col].fillna(0).melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

c:\Users\gidon\AppData\Local\anaconda3\Lib\site-packages\geopandas\geodataframe.py:2469: UserWarning: `keep_geom_type=True` in overlay resulted in 13 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [39]:
Jewish_kibolt_some_by_plan['source']='by_plan'

### מכאן  צריך לחבר ביחד את כל הטבלאות

In [40]:
kibolt_jew=pd.concat([Jewish_kibolt_some_by_plan,Jewish_kibolt_some_by_plan_only_with_taz,kibolt_yosh_some_towns_sum_by_taz,Jewish_kibolt_some_by_taz],axis=0)

In [41]:
kibolt_jew['year']=kibolt_jew['year'].astype(str)

In [42]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [43]:
kibolt_jew.to_excel(r'{}\Intermediates\{}_kibolt_jew_2050_by_type.xlsx'.format(software_data_folder_location,file_date),index=False)

### ייצוא לקלט של מכונת התפלגות גילים תרחיש צתאל

In [44]:
kibolt_jew_jtmt=kibolt_jew.pivot_table(index='Taz_num',columns='year',values='add_aprt',aggfunc=sum).fillna(0).reset_index()

old_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050']

new_col=[ 'Taz_num','add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

kibolt_jew_jtmt=kibolt_jew_jtmt[old_col]

kibolt_jew_jtmt.columns=new_col



kibolt_jew_jtmt.to_excel(r'{}\Intermediates\{}_kibolt_jew_till_2050.xlsx'.format(software_data_folder_location,file_date),index=False)